In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sb 
import numpy_financial as npf
from sklearn.metrics import confusion_matrix, accuracy_score 

In [2]:
df= pd.read_csv('downloads/ProsperLoanData.csv')
df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,09:29.3,C,36,Completed,8/14/2009 0:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,28:07.9,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,00:47.1,HR,36,Completed,12/17/2009 0:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,02:35.0,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,38:39.1,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


In [3]:
df1 = df[['BorrowerRate','EstimatedReturn','ProsperRating (numeric)','ListingCategory (numeric)',' IsBorrowerHomeowner','CreditScoreRangeLower',
         'OpenCreditLines','InquiriesLast6Months','PublicRecordsLast12Months','BankcardUtilization','DebtToIncomeRatio','StatedMonthlyIncome','LoanOriginalAmount','LoanOriginationDate','LoanStatus']]

In [4]:
import datetime

In [5]:
df1['Datetime']= pd.to_datetime(df1['LoanOriginationDate'])


C:\Users\JAHANL~1\AppData\Local\Temp/ipykernel_12100/199776526.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Datetime']= pd.to_datetime(df1['LoanOriginationDate'])


In [6]:
cutoff_date ='2009-12-31'
df1= df1.loc[df1['Datetime'] >= cutoff_date]

In [7]:
df1= df1.drop(['Datetime'],axis=1)


In [8]:
df1.drop(df1.index[np.where(df1['LoanStatus'] == 'Current')], inplace=True)
df1.drop(df1.index[np.where(df1['LoanStatus'] == 'Past Due (1-15 days)')], inplace=True)
df1.drop(df1.index[np.where(df1['LoanStatus'] == 'Past Due (16-30 days)')], inplace=True)
df1.drop(df1.index[np.where(df1['LoanStatus'] == 'Past Due (61-90 days)')], inplace=True)
df1.drop(df1.index[np.where(df1['LoanStatus'] == 'Past Due (31-60 days)')], inplace=True)
df1.drop(df1.index[np.where(df1['LoanStatus'] == 'Past Due (91-120 days)')], inplace=True)
df1.drop(df1.index[np.where(df1['LoanStatus'] == 'Past Due (>120 days)')], inplace=True)
df1.drop(df1.index[np.where(df1['LoanStatus'] == 'FinalPaymentInProgress')], inplace=True)


In [9]:
df1

,BorrowerRate,EstimatedReturn,ProsperRating (numeric),ListingCategory (numeric),IsBorrowerHomeowner,CreditScoreRangeLower,OpenCreditLines,InquiriesLast6Months,PublicRecordsLast12Months,BankcardUtilization,DebtToIncomeRatio,StatedMonthlyIncome,LoanOriginalAmount,LoanOriginationDate,LoanStatus
15,0.3177,0.1246,1.0,13,True,700.0,9.0,0.0,0.0,0.97,0.49,5500.000000,4000,4/19/2012 0:00,Defaulted
23,0.2419,0.1051,3.0,15,False,680.0,7.0,0.0,0.0,0.84,0.39,2500.000000,2000,4/26/2013 0:00,Defaulted
26,0.2809,0.1248,2.0,1,False,660.0,6.0,2.0,0.0,0.30,0.11,3885.916667,4000,2/21/2013 0:00,Completed
27,0.0920,0.0608,6.0,1,True,700.0,16.0,2.0,0.0,0.09,0.26,6666.666667,4000,6/24/2010 0:00,Completed
30,0.0999,0.0517,6.0,20,False,740.0,4.0,0.0,0.0,0.13,0.11,3750.000000,10000,2/7/2012 0:00,Completed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113908,0.3220,0.1557,2.0,3,False,700.0,15.0,1.0,0.0,0.00,NaN,2833.333333,7500,12/8/2010 0:00,Completed
113913,0.1030,0.0717,6.0,1,True,780.0,19.0,0.0,0.0,0.59,0.25,9750.000000,8000,5/5/2010 0:00,Completed
113928,0.1899,0.1147,4.0,3,False,740.0,3.0,0.0,0.0,0.00,0.27,2333.333333,2000,6/10/2011 0:00,Completed
113929,0.2639,0.1093,2.0,2,False,660.0,7.0,2.0,0.0,0.11,0.05,4333.333333,2500,7/10/2013 0:00,Completed


In [10]:
from scipy.stats import spearmanr


In [11]:
x=np.array(df1['BorrowerRate'])
x

array([0.3177, 0.2419, 0.2809, ..., 0.1899, 0.2639, 0.2605])

In [12]:
y=np.array(df1['EstimatedReturn'])
y.shape

(24176,)

In [13]:
df1

,BorrowerRate,EstimatedReturn,ProsperRating (numeric),ListingCategory (numeric),IsBorrowerHomeowner,CreditScoreRangeLower,OpenCreditLines,InquiriesLast6Months,PublicRecordsLast12Months,BankcardUtilization,DebtToIncomeRatio,StatedMonthlyIncome,LoanOriginalAmount,LoanOriginationDate,LoanStatus
15,0.3177,0.1246,1.0,13,True,700.0,9.0,0.0,0.0,0.97,0.49,5500.000000,4000,4/19/2012 0:00,Defaulted
23,0.2419,0.1051,3.0,15,False,680.0,7.0,0.0,0.0,0.84,0.39,2500.000000,2000,4/26/2013 0:00,Defaulted
26,0.2809,0.1248,2.0,1,False,660.0,6.0,2.0,0.0,0.30,0.11,3885.916667,4000,2/21/2013 0:00,Completed
27,0.0920,0.0608,6.0,1,True,700.0,16.0,2.0,0.0,0.09,0.26,6666.666667,4000,6/24/2010 0:00,Completed
30,0.0999,0.0517,6.0,20,False,740.0,4.0,0.0,0.0,0.13,0.11,3750.000000,10000,2/7/2012 0:00,Completed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113908,0.3220,0.1557,2.0,3,False,700.0,15.0,1.0,0.0,0.00,NaN,2833.333333,7500,12/8/2010 0:00,Completed
113913,0.1030,0.0717,6.0,1,True,780.0,19.0,0.0,0.0,0.59,0.25,9750.000000,8000,5/5/2010 0:00,Completed
113928,0.1899,0.1147,4.0,3,False,740.0,3.0,0.0,0.0,0.00,0.27,2333.333333,2000,6/10/2011 0:00,Completed
113929,0.2639,0.1093,2.0,2,False,660.0,7.0,2.0,0.0,0.11,0.05,4333.333333,2500,7/10/2013 0:00,Completed


In [14]:
df1=df1.dropna(subset=['EstimatedReturn', 'DebtToIncomeRatio', 'ProsperRating (numeric)'])

In [15]:
df1.columns


Index(['BorrowerRate', 'EstimatedReturn', 'ProsperRating (numeric)',
       'ListingCategory (numeric)', ' IsBorrowerHomeowner',
       'CreditScoreRangeLower', 'OpenCreditLines', 'InquiriesLast6Months',
       'PublicRecordsLast12Months', 'BankcardUtilization', 'DebtToIncomeRatio',
       'StatedMonthlyIncome', 'LoanOriginalAmount', 'LoanOriginationDate',
       'LoanStatus'],
      dtype='object')

In [16]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['BorrowerRate'])
corr, pval

(1.0, 0.0)

In [17]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['EstimatedReturn'])


In [18]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['BorrowerRate'])

In [19]:
df1.columns

Index(['BorrowerRate', 'EstimatedReturn', 'ProsperRating (numeric)',
       'ListingCategory (numeric)', ' IsBorrowerHomeowner',
       'CreditScoreRangeLower', 'OpenCreditLines', 'InquiriesLast6Months',
       'PublicRecordsLast12Months', 'BankcardUtilization', 'DebtToIncomeRatio',
       'StatedMonthlyIncome', 'LoanOriginalAmount', 'LoanOriginationDate',
       'LoanStatus'],
      dtype='object')

In [20]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['BorrowerRate'])
corr, pval

(1.0, 0.0)

In [21]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['EstimatedReturn'])
corr, pval

(0.8030520074856639, 0.0)

In [22]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['ProsperRating (numeric)'])
corr, pval

(-0.9491051166540087, 0.0)

In [23]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['ListingCategory (numeric)'])
corr, pval

(0.02259556842986406, 0.00097128022158398)

In [24]:
corr, pval = spearmanr(df1['BorrowerRate'],df1[' IsBorrowerHomeowner'])
corr, pval

(-0.07284156057192907, 1.8028394060235885e-26)

In [25]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['CreditScoreRangeLower'])
corr, pval

(-0.5625233323137722, 0.0)

In [26]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['OpenCreditLines'])
corr, pval

(-0.07147294537874851, 1.5230501773973718e-25)

In [27]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['InquiriesLast6Months'])
corr, pval

(0.2905358338572551, 0.0)

In [28]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['PublicRecordsLast12Months'])
corr, pval

(0.04172583057336635, 1.1053041555339495e-09)

In [29]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['BankcardUtilization'])
corr, pval

(0.3103943372425455, 0.0)

In [30]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['DebtToIncomeRatio'])
corr, pval

(0.24687272385618875, 2.451764421208164e-293)

In [31]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['StatedMonthlyIncome'])
corr, pval

(-0.22583580504281536, 1.4753350004574345e-244)

In [32]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['LoanOriginalAmount'])
corr, pval

(-0.2777283875998925, 0.0)

In [33]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['LoanStatus'])
corr, pval

(-0.18543019737040436, 3.748367684295453e-164)

In [34]:
corr, pval = spearmanr(df1['EstimatedReturn'],df1['LoanStatus'])
corr, pval

(-0.17353945302760213, 1.0246708639027424e-143)

In [35]:
corr, pval = spearmanr(df1['EstimatedReturn'],df1['LoanOriginalAmount'])
corr, pval

(-0.06489620816581985, 2.4718483071100797e-21)

In [36]:
corr, pval = spearmanr(df1['EstimatedReturn'],df1['StatedMonthlyIncome'])
corr, pval

(-0.17646932171632115, 1.2839455499511591e-148)

In [37]:
corr, pval = spearmanr(df1['EstimatedReturn'],df1['DebtToIncomeRatio'])
corr, pval

(0.19426414445487744, 2.83967990022086e-180)

In [38]:
corr, pval = spearmanr(df1['EstimatedReturn'],df1['BankcardUtilization'])
corr, pval

(0.24931726058630427, 2.539719884610263e-299)

In [39]:
corr, pval = spearmanr(df1['EstimatedReturn'],df1['PublicRecordsLast12Months'])
corr, pval

(0.03670084640576249, 8.365586791397622e-08)

In [40]:
corr, pval = spearmanr(df1['EstimatedReturn'],df1['InquiriesLast6Months'])
corr, pval

(0.1754068469430818, 7.875104458983212e-147)

In [41]:
corr, pval = spearmanr(df1['EstimatedReturn'],df1['OpenCreditLines'])
corr, pval

(-0.058554299434852214, 1.183161033219067e-17)

In [42]:
corr, pval = spearmanr(df1['EstimatedReturn'],df1['CreditScoreRangeLower'])
corr, pval

(-0.42842608420219824, 0.0)

In [43]:
corr, pval = spearmanr(df1['EstimatedReturn'],df1[' IsBorrowerHomeowner'])
corr, pval

(-0.05854633896307781, 1.1951691660050916e-17)

In [44]:
corr, pval = spearmanr(df1['EstimatedReturn'],df1['ListingCategory (numeric)'])
corr, pval

(0.000714276086519476, 0.9169602208871768)

In [45]:
corr, pval = spearmanr(df1['EstimatedReturn'],df1['ProsperRating (numeric)'])
corr, pval

(-0.6415676281093213, 0.0)

In [46]:
corr, pval = spearmanr(df1['BorrowerRate'],df1['EstimatedReturn'])
corr, pval

(0.8030520074856639, 0.0)

In [47]:
df1['BorrowerRate']

15        0.3177
23        0.2419
26        0.2809
27        0.0920
30        0.0999
           ...  
113905    0.3177
113913    0.1030
113928    0.1899
113929    0.2639
113935    0.2605
Name: BorrowerRate, Length: 21310, dtype: float64

In [48]:
rankedinterest= df1['BorrowerRate'].rank()

In [49]:
rankedinterest

15        18403.5
23        11466.5
26        15477.5
27         1873.5
30         2612.5
           ...   
113905    18403.5
113913     2949.5
113928     8193.0
113929    13909.0
113935    13553.0
Name: BorrowerRate, Length: 21310, dtype: float64

In [50]:
rankedirr= df1['EstimatedReturn'].rank()

In [51]:
rankedirr

15        13786.5
23         9130.5
26        14241.5
27         2909.0
30         1320.0
           ...   
113905    13786.5
113913     4471.5
113928    11215.5
113929    10087.0
113935    20557.5
Name: EstimatedReturn, Length: 21310, dtype: float64

In [52]:
df1.insert( 15,'r_IRR', rankedirr, True)

In [53]:
df1.insert( 16,'r_interes trate',rankedinterest , True)

In [54]:
df1

,BorrowerRate,EstimatedReturn,ProsperRating (numeric),ListingCategory (numeric),IsBorrowerHomeowner,CreditScoreRangeLower,OpenCreditLines,InquiriesLast6Months,PublicRecordsLast12Months,BankcardUtilization,DebtToIncomeRatio,StatedMonthlyIncome,LoanOriginalAmount,LoanOriginationDate,LoanStatus,r_IRR,r_interes trate
15,0.3177,0.1246,1.0,13,True,700.0,9.0,0.0,0.0,0.97,0.49,5500.000000,4000,4/19/2012 0:00,Defaulted,13786.5,18403.5
23,0.2419,0.1051,3.0,15,False,680.0,7.0,0.0,0.0,0.84,0.39,2500.000000,2000,4/26/2013 0:00,Defaulted,9130.5,11466.5
26,0.2809,0.1248,2.0,1,False,660.0,6.0,2.0,0.0,0.30,0.11,3885.916667,4000,2/21/2013 0:00,Completed,14241.5,15477.5
27,0.0920,0.0608,6.0,1,True,700.0,16.0,2.0,0.0,0.09,0.26,6666.666667,4000,6/24/2010 0:00,Completed,2909.0,1873.5
30,0.0999,0.0517,6.0,20,False,740.0,4.0,0.0,0.0,0.13,0.11,3750.000000,10000,2/7/2012 0:00,Completed,1320.0,2612.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113905,0.3177,0.1246,1.0,1,True,740.0,16.0,3.0,0.0,0.75,0.35,3250.000000,4000,12/5/2011 0:00,Chargedoff,13786.5,18403.5
113913,0.1030,0.0717,6.0,1,True,780.0,19.0,0.0,0.0,0.59,0.25,9750.000000,8000,5/5/2010 0:00,Completed,4471.5,2949.5
113928,0.1899,0.1147,4.0,3,False,740.0,3.0,0.0,0.0,0.00,0.27,2333.333333,2000,6/10/2011 0:00,Completed,11215.5,8193.0
113929,0.2639,0.1093,2.0,2,False,660.0,7.0,2.0,0.0,0.11,0.05,4333.333333,2500,7/10/2013 0:00,Completed,10087.0,13909.0


In [55]:
label = []

for i in range(len(df1)):
    if df1.iloc[i][' IsBorrowerHomeowner'] == True :
        label.append(1)
    else:
        label.append(0) 

In [56]:
df1.insert(12, 'Home', label , True)

In [57]:
df1=df1.drop([' IsBorrowerHomeowner'], axis=1)

In [58]:
labels = []

for i in range(len(df1)):
    if df1.iloc[i]['LoanStatus'] == 'Completed' :
        labels.append(1)
    else:
        labels.append(0)

In [59]:
df1.insert(12, 'Loanstatu', labels , True)

In [60]:
df1=df1.drop(['LoanStatus'], axis=1)

In [61]:
df1

,BorrowerRate,EstimatedReturn,ProsperRating (numeric),ListingCategory (numeric),CreditScoreRangeLower,OpenCreditLines,InquiriesLast6Months,PublicRecordsLast12Months,BankcardUtilization,DebtToIncomeRatio,StatedMonthlyIncome,Home,Loanstatu,LoanOriginalAmount,LoanOriginationDate,r_IRR,r_interes trate
15,0.3177,0.1246,1.0,13,700.0,9.0,0.0,0.0,0.97,0.49,5500.000000,1,0,4000,4/19/2012 0:00,13786.5,18403.5
23,0.2419,0.1051,3.0,15,680.0,7.0,0.0,0.0,0.84,0.39,2500.000000,0,0,2000,4/26/2013 0:00,9130.5,11466.5
26,0.2809,0.1248,2.0,1,660.0,6.0,2.0,0.0,0.30,0.11,3885.916667,0,1,4000,2/21/2013 0:00,14241.5,15477.5
27,0.0920,0.0608,6.0,1,700.0,16.0,2.0,0.0,0.09,0.26,6666.666667,1,1,4000,6/24/2010 0:00,2909.0,1873.5
30,0.0999,0.0517,6.0,20,740.0,4.0,0.0,0.0,0.13,0.11,3750.000000,0,1,10000,2/7/2012 0:00,1320.0,2612.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113905,0.3177,0.1246,1.0,1,740.0,16.0,3.0,0.0,0.75,0.35,3250.000000,1,0,4000,12/5/2011 0:00,13786.5,18403.5
113913,0.1030,0.0717,6.0,1,780.0,19.0,0.0,0.0,0.59,0.25,9750.000000,1,1,8000,5/5/2010 0:00,4471.5,2949.5
113928,0.1899,0.1147,4.0,3,740.0,3.0,0.0,0.0,0.00,0.27,2333.333333,0,1,2000,6/10/2011 0:00,11215.5,8193.0
113929,0.2639,0.1093,2.0,2,660.0,7.0,2.0,0.0,0.11,0.05,4333.333333,0,1,2500,7/10/2013 0:00,10087.0,13909.0


In [62]:
df1.isnull().sum()

BorrowerRate                 0
EstimatedReturn              0
ProsperRating (numeric)      0
ListingCategory (numeric)    0
CreditScoreRangeLower        0
OpenCreditLines              0
InquiriesLast6Months         0
PublicRecordsLast12Months    0
BankcardUtilization          0
DebtToIncomeRatio            0
StatedMonthlyIncome          0
Home                         0
Loanstatu                    0
LoanOriginalAmount           0
LoanOriginationDate          0
r_IRR                        0
r_interes trate              0
dtype: int64

In [63]:
from sklearn.utils import shuffle

In [64]:
df2 = shuffle(df1)

In [65]:
df2

,BorrowerRate,EstimatedReturn,ProsperRating (numeric),ListingCategory (numeric),CreditScoreRangeLower,OpenCreditLines,InquiriesLast6Months,PublicRecordsLast12Months,BankcardUtilization,DebtToIncomeRatio,StatedMonthlyIncome,Home,Loanstatu,LoanOriginalAmount,LoanOriginationDate,r_IRR,r_interes trate
77298,0.2254,0.0682,2.0,16,680.0,2.0,3.0,0.0,0.79,0.11,5833.333333,0,1,2000,9/17/2012 0:00,4080.0,10609.5
59522,0.2669,0.1067,2.0,1,660.0,9.0,2.0,0.0,0.68,0.49,2916.666667,0,1,6000,11/8/2011 0:00,9318.5,13989.0
5988,0.2999,0.1276,2.0,1,640.0,9.0,0.0,0.0,0.66,0.29,3291.666667,0,0,4000,3/8/2012 0:00,14860.0,16419.5
55726,0.2610,0.1458,3.0,3,620.0,3.0,5.0,0.0,0.51,0.19,3333.333333,0,0,11000,6/26/2012 0:00,18264.5,13596.0
27624,0.1110,0.0807,6.0,3,680.0,5.0,0.0,0.0,0.51,0.18,5416.666667,0,1,4000,11/7/2011 0:00,5901.5,3449.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75118,0.1493,0.0548,4.0,20,640.0,6.0,0.0,0.0,0.81,0.12,6166.666667,0,1,2000,11/5/2012 0:00,1647.0,5723.0
92700,0.1380,0.0759,5.0,1,720.0,9.0,1.0,0.0,0.13,0.23,4416.666667,1,1,8780,3/23/2010 0:00,4939.5,4549.5
6407,0.3099,0.1561,2.0,2,640.0,6.0,4.0,0.0,0.82,0.14,10000.000000,0,1,7500,9/22/2011 0:00,20212.5,16895.0
82899,0.2839,0.1239,2.0,1,640.0,10.0,0.0,0.0,0.73,0.12,5666.666667,1,1,4000,9/13/2012 0:00,13093.0,15502.5


In [66]:
from sklearn.model_selection import train_test_split

In [67]:
df2.columns

Index(['BorrowerRate', 'EstimatedReturn', 'ProsperRating (numeric)',
       'ListingCategory (numeric)', 'CreditScoreRangeLower', 'OpenCreditLines',
       'InquiriesLast6Months', 'PublicRecordsLast12Months',
       'BankcardUtilization', 'DebtToIncomeRatio', 'StatedMonthlyIncome',
       'Home', 'Loanstatu', 'LoanOriginalAmount', 'LoanOriginationDate',
       'r_IRR', 'r_interes trate'],
      dtype='object')

In [68]:
#x= df2.drop(['r_interes trate','Loanstatu','EstimatedReturn','LoanOriginationDate'], axis=1)
x= df2[['BorrowerRate','ProsperRating (numeric)',
       'ListingCategory (numeric)', 'CreditScoreRangeLower', 'OpenCreditLines',
       'InquiriesLast6Months', 'PublicRecordsLast12Months',
       'BankcardUtilization', 'DebtToIncomeRatio', 'StatedMonthlyIncome',
       'Home','LoanOriginalAmount']]

In [69]:
x

,BorrowerRate,ProsperRating (numeric),ListingCategory (numeric),CreditScoreRangeLower,OpenCreditLines,InquiriesLast6Months,PublicRecordsLast12Months,BankcardUtilization,DebtToIncomeRatio,StatedMonthlyIncome,Home,LoanOriginalAmount
77298,0.2254,2.0,16,680.0,2.0,3.0,0.0,0.79,0.11,5833.333333,0,2000
59522,0.2669,2.0,1,660.0,9.0,2.0,0.0,0.68,0.49,2916.666667,0,6000
5988,0.2999,2.0,1,640.0,9.0,0.0,0.0,0.66,0.29,3291.666667,0,4000
55726,0.2610,3.0,3,620.0,3.0,5.0,0.0,0.51,0.19,3333.333333,0,11000
27624,0.1110,6.0,3,680.0,5.0,0.0,0.0,0.51,0.18,5416.666667,0,4000
...,...,...,...,...,...,...,...,...,...,...,...,...
75118,0.1493,4.0,20,640.0,6.0,0.0,0.0,0.81,0.12,6166.666667,0,2000
92700,0.1380,5.0,1,720.0,9.0,1.0,0.0,0.13,0.23,4416.666667,1,8780
6407,0.3099,2.0,2,640.0,6.0,4.0,0.0,0.82,0.14,10000.000000,0,7500
82899,0.2839,2.0,1,640.0,10.0,0.0,0.0,0.73,0.12,5666.666667,1,4000


In [70]:
y= df2['EstimatedReturn']

In [71]:
y

77298    0.0682
59522    0.1067
5988     0.1276
55726    0.1458
27624    0.0807
          ...  
75118    0.0548
92700    0.0759
6407     0.1561
82899    0.1239
81072    0.0708
Name: EstimatedReturn, Length: 21310, dtype: float64

In [72]:
x_train, x_test , y_train, y_test = train_test_split(x, y , train_size=.7)

In [73]:
from sklearn import linear_model

In [74]:
from sklearn.linear_model import LinearRegression

In [75]:
model1= LinearRegression().fit(x_train, y_train)

In [76]:
pred1= model1.predict(x_test)
pred1

array([0.12068302, 0.07911619, 0.1532285 , ..., 0.06737144, 0.15133537,
       0.15398892])

In [77]:
pred1.shape

(6394,)

In [78]:
score= model1.score(x_test, y_test)
score

0.8186147774421383

In [79]:
model1.coef_  

array([ 9.17502490e-01,  2.85613000e-02, -1.92698950e-05, -4.30642687e-05,
        5.17322572e-05, -1.13713956e-03, -1.81281805e-03,  8.85437943e-04,
        2.93144935e-04, -5.59718555e-08, -3.78205827e-04,  1.20110531e-07])

In [80]:
model1.coef_

array([ 9.17502490e-01,  2.85613000e-02, -1.92698950e-05, -4.30642687e-05,
        5.17322572e-05, -1.13713956e-03, -1.81281805e-03,  8.85437943e-04,
        2.93144935e-04, -5.59718555e-08, -3.78205827e-04,  1.20110531e-07])

In [81]:
x2=df2[['r_interes trate','ProsperRating (numeric)','CreditScoreRangeLower']]

In [82]:
x2

,r_interes trate,ProsperRating (numeric),CreditScoreRangeLower
77298,10609.5,2.0,680.0
59522,13989.0,2.0,660.0
5988,16419.5,2.0,640.0
55726,13596.0,3.0,620.0
27624,3449.0,6.0,680.0
...,...,...,...
75118,5723.0,4.0,640.0
92700,4549.5,5.0,720.0
6407,16895.0,2.0,640.0
82899,15502.5,2.0,640.0


In [83]:
df2.columns

Index(['BorrowerRate', 'EstimatedReturn', 'ProsperRating (numeric)',
       'ListingCategory (numeric)', 'CreditScoreRangeLower', 'OpenCreditLines',
       'InquiriesLast6Months', 'PublicRecordsLast12Months',
       'BankcardUtilization', 'DebtToIncomeRatio', 'StatedMonthlyIncome',
       'Home', 'Loanstatu', 'LoanOriginalAmount', 'LoanOriginationDate',
       'r_IRR', 'r_interes trate'],
      dtype='object')

In [84]:
y2= df2['r_IRR']

In [85]:
y2

77298     4080.0
59522     9318.5
5988     14860.0
55726    18264.5
27624     5901.5
          ...   
75118     1647.0
92700     4939.5
6407     20212.5
82899    13093.0
81072     4380.5
Name: r_IRR, Length: 21310, dtype: float64

In [86]:
x2_train, x2_test , y2_train, y2_test = train_test_split(x2, y2, train_size=.7)

In [87]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression


In [88]:
model2= LinearRegression().fit(x2_train, y2_train)
pred2= model2.predict(x2_test)
pred2

array([ 8905.24054186,  5346.36831876,  9009.32194436, ...,
       21679.90833535, 11078.86330638, 14387.64213701])

In [89]:
x2_train

,r_interes trate,ProsperRating (numeric),CreditScoreRangeLower
27211,8625.0,4.0,680.0
4855,19599.5,2.0,660.0
97476,9202.5,4.0,740.0
12489,6762.0,5.0,660.0
113846,17549.0,2.0,680.0
...,...,...,...
65804,16895.0,2.0,600.0
39855,9752.5,4.0,680.0
62012,3992.5,6.0,800.0
90614,15887.0,2.0,680.0


In [90]:
y2_train

27211     11698.0
4855      18871.0
97476      8658.5
12489      7569.0
113846    17793.0
           ...   
65804     20212.5
39855     10782.5
62012      4611.5
90614     14284.0
10408     16179.0
Name: r_IRR, Length: 14916, dtype: float64

In [91]:
score2= model2.score(x2_test, y2_test)
score2

0.7731226447078797

In [92]:
model2.coef_

array([ 1.83549123e+00,  3.95861298e+03, -1.26907191e+01])

In [93]:
df2.columns


Index(['BorrowerRate', 'EstimatedReturn', 'ProsperRating (numeric)',
       'ListingCategory (numeric)', 'CreditScoreRangeLower', 'OpenCreditLines',
       'InquiriesLast6Months', 'PublicRecordsLast12Months',
       'BankcardUtilization', 'DebtToIncomeRatio', 'StatedMonthlyIncome',
       'Home', 'Loanstatu', 'LoanOriginalAmount', 'LoanOriginationDate',
       'r_IRR', 'r_interes trate'],
      dtype='object')

In [94]:
x3= df2[['ListingCategory (numeric)', 'LoanOriginalAmount','r_interes trate']]

In [95]:
y3=df2['r_IRR']

In [96]:
x3_train, x3_test , y3_train, y3_test = train_test_split(x3, y3, train_size=.7)

In [97]:
model3= LinearRegression().fit(x3_train, y3_train)
pred3= model3.predict(x3_test)
pred3

array([14042.93969287,  8237.02013527, 15222.76415797, ...,
        5619.93807172, 12017.27244128,  2866.36339302])

In [98]:
score3= model3.score(x3_test, y3_test)
score3

0.6633286740142103

In [99]:
model3.coef_

array([-8.95876285,  0.18020517,  0.84763228])

In [100]:
df2.columns

Index(['BorrowerRate', 'EstimatedReturn', 'ProsperRating (numeric)',
       'ListingCategory (numeric)', 'CreditScoreRangeLower', 'OpenCreditLines',
       'InquiriesLast6Months', 'PublicRecordsLast12Months',
       'BankcardUtilization', 'DebtToIncomeRatio', 'StatedMonthlyIncome',
       'Home', 'Loanstatu', 'LoanOriginalAmount', 'LoanOriginationDate',
       'r_IRR', 'r_interes trate'],
      dtype='object')

In [101]:
x4= df2[['Home','StatedMonthlyIncome','r_interes trate']]

In [102]:
y4= df2['r_IRR']

In [103]:
x4_train, x4_test , y4_train, y4_test = train_test_split(x4, y4, train_size=.7)

In [104]:
model4= LinearRegression().fit(x4_train, y4_train)
pred4= model4.predict(x4_test)
pred4

array([15624.93454   , 14272.0341611 ,  8511.33957419, ...,
       17806.90819167,  8435.90784118, 11170.30251297])

In [105]:
score4= model4.score(x4_test, y4_test)
score4

0.6507202476135383

In [106]:
model4.coef_

array([3.39851992, 0.01048877, 0.80420822])

In [107]:
x5= df2[['InquiriesLast6Months', 'PublicRecordsLast12Months','OpenCreditLines','BankcardUtilization','r_interes trate']]

In [108]:
y5= df2['r_IRR']

In [109]:
x5_train, x5_test , y5_train, y5_test = train_test_split(x5, y5, train_size=.7)

In [110]:
model5= LinearRegression().fit(x5_train, y5_train)
pred5= model5.predict(x5_test)
pred5

array([15833.85976053,  6290.61757157, 10516.99424869, ...,
       15629.55372958, 18017.73491117,  7184.36617812])

In [111]:
score5= model5.score(x5_test, y5_test)
score5

0.6501697162880435

In [112]:
model5.coef_

array([-295.22756506,   96.97409672,   10.94584937, -261.25840305,
          0.83317148])

In [113]:
x6= df2[['ProsperRating (numeric)',
       'ListingCategory (numeric)', 'CreditScoreRangeLower', 'OpenCreditLines',
       'InquiriesLast6Months', 'PublicRecordsLast12Months',
       'BankcardUtilization', 'DebtToIncomeRatio', 'StatedMonthlyIncome',
       'Home', 'LoanOriginalAmount','r_interes trate']]

In [114]:
y6= df2['r_IRR']

In [115]:
x6_train, x6_test , y6_train, y6_test = train_test_split(x6, y6, train_size=.7)

In [116]:
model6= LinearRegression().fit(x6_train, y6_train)
pred6= model6.predict(x6_test)
pred6

array([ 2576.2210239 , 16600.81923178, 15259.82419794, ...,
        3024.98843376, 20203.69770196, 13219.49980505])

In [117]:
score6= model6.score(x6_test, y6_test)
score6

0.786149358289868

In [118]:
model6.coef_

array([ 3.74969635e+03,  4.14982700e+00, -1.18875108e+01,  2.52160329e+00,
       -1.29457501e+02, -7.56996091e+01,  3.04789098e+02,  4.01812352e+01,
       -1.97285156e-02, -3.91329628e+01,  7.50358666e-02,  1.79931816e+00])